In [ ]:
import sys
sys.path.insert(0, '../')

import utils, evaluation
#from evaluation import load_gan
from data import fmap, Dataset
from JR_Scripts import dict_reader, time_toy_generator
from JR_Scripts import notebook_helper as nh
from model import WGanModel, WNGanModel, TemporalGanModelv3
from gan import TimeCosmoGAN
import metrics
import tensorflow as tf
import numpy as np
from PIL import Image
import imageio
from matplotlib.pyplot import imshow
import matplotlib.pyplot as plt
import matplotlib.cm as cm
import importlib as imp

In [ ]:
redshift = ['z = 0.000', 'z = 0.111', 'z = 0.250', 'z = 0.428', 'z = 0.666', 'z = 1.000', 'z = 1.500', 'z = 2.333', 'z = 4.000', 'z = 9.000']

In [ ]:
def load_gan_and_gen_data_in_old_format(model_folder, specific_model, params):
    gan = evaluation.load_gan(model_folder, GANtype=TimeCosmoGAN)
    t = params['time']['classes']
    images = nh.gen_fake_images(gan, 5000, t, checkpoint=specific_model)
    return params['cosmology']['backward_map'](images)

In [ ]:
data_lst = []
labels_lst = ['Specialized', 'CCGAN', 'Real']
colors_lst = ['b', 'g', 'r']

In [ ]:
params, model_folder = nh.load_params("models_64x64_C4/TWGANce:62d3_selu_sn10-10_0246r_Hlr3e5_bs4_v2ad_c+sf62_checkpoints/", cscs=True)
specific_model = 164800

In [ ]:
nh.print_param_dict(params)

In [ ]:
ccgan_images = load_gan_and_gen_data_in_old_format(model_folder, specific_model, params)

In [ ]:
dset = nh.get_dataset(500, 512, params, shuffle=True, cscs=True)

In [ ]:
#nh.get_scaling(gan.params['time']['classes'], gan.params['time']['class_weights'])

In [ ]:
data = dset.get_samples(5000)
data = nh.reshape_data_to_old_format(data, params)
data = params['cosmology']['backward_map'](data)
data.shape

In [ ]:
#data_lst.append(data)

In [ ]:
def create_curves(data, ccgan_images, labels_lst, colors_lst, t, nc, special_model_folder, specific_model):
    params, model_folder = nh.load_params(special_model_folder, cscs=True)
    data_lst = []
    data_lst.append(load_gan_and_gen_data_in_old_format(model_folder, specific_model, params))
    data_lst.append(ccgan_images[t::nc])
    data_lst.append(data[t::nc])
    
    lim_mass = nh.get_lim_mass(data_lst[2])
    nh.plot_mass_hists(data_lst, labels_lst, colors_lst, lim_mass, params)
    
    _, _, lim = metrics.peak_count_hist(data_lst[1])
    lim = (lim[0], lim[1] + 0.5)
    nh.plot_peak_hists(data_lst, labels_lst, colors_lst, lim, params)
    
    nh.plot_power_spectral_densities(data_lst, labels_lst, colors_lst, params)

In [ ]:
for ts in [0, 2, 4, 6]:
    special_model_folder = "models_64x64/TWGANv2:62d3_lap_selu-sn6-5_4Mom_{}r_CDF62_checkpoints/".format(ts)
    specific_model = 150000
    create_curves(data, ccgan_images, labels_lst, colors_lst, ts // 2, 4, special_model_folder, specific_model)